In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def mediapipe_Detection(image, model):
      #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #set flag to False
        image.flags.writeable = False
        #Detection 
        results = model.process(image)
        #RGB to BGR
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        #set flag to True
        image.flags.writeable = True

        return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
         mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
         mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
         mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=2 ))

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read Feed
        ret, frame = cap.read()

        image, results = mediapipe_Detection(frame, holistic)
        draw_landmarks(frame,results)
        #show to screen
        cv2.imshow('Webcam Feed', frame)

        #end loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [7]:
#path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# actions list
actions = np.array(['hello', 'thanks', 'iloveyou']) 
# actions = np.array(['hello']) 

#30 videos of data
no_sequence = 30

# 30 frames each
sequence_length = 30

In [11]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [12]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #loop through actions
    for action in actions:
        #loop through sequences aka videos
        for sequence in range(no_sequence):
            #loop through frames
            for frame_num in range(sequence_length):
                #Read Feed
                ret, frame = cap.read()

                image, results = mediapipe_Detection(frame, holistic)
                draw_landmarks(image,results)

                #apply wait logic
                if frame_num == 0:
                    cv2.putText(image,'STARTING_COLLECTION',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA)
                    cv2.waitKey(500)
                    cv2.putText(image,'Collecting Frames for {} video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,
                    (0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(500)
                else:
                    cv2.putText(image,'Collecting Frames for {} video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,
                    (0,0,255),1,cv2.LINE_AA)

                #Export Keypoints
                keypoints = extract_keypoints(results)
                np_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(np_path,keypoints)
                 
                #show to screen
                cv2.imshow('Webcam Feed', image)

                #end loop
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

cap.release()
cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequence):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
x= np.array(sequences) 

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True,activation='relu'))
model.add(LSTM(64, return_sequences=False,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [21]:
res = [.7,0.2,0.1]

In [22]:
actions[np.argmax(res)]

'hello'

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [24]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 74ms/step - loss: 1.1627 - categorical_accuracy: 0.3059
Epoch 2/2000
3/3 [==============================] - 0s 118ms/step - loss: 3.0635 - categorical_accuracy: 0.3529
Epoch 3/2000
3/3 [==============================] - 0s 76ms/step - loss: 1.2434 - categorical_accuracy: 0.2941
Epoch 4/2000
3/3 [==============================] - 0s 73ms/step - loss: 1.1368 - categorical_accuracy: 0.3765
Epoch 5/2000
3/3 [==============================] - 0s 79ms/step - loss: 1.1664 - categorical_accuracy: 0.3176
Epoch 6/2000
3/3 [==============================] - 0s 74ms/step - loss: 1.1100 - categorical_accuracy: 0.3529
Epoch 7/2000
3/3 [==============================] - 0s 74ms/step - loss: 1.0059 - categorical_accuracy: 0.5176
Epoch 8/2000
3/3 [==============================] - 0s 87ms/step - loss: 1.0788 - categorical_accuracy: 0.3647
Epoch 9/2000
3/3 [==============================] - 0s 71ms/step - loss: 1.3947 - categorical_accuracy: 0.4588


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [26]:
res = model.predict(x_test)

1/1 [==============================] - 0s 379ms/step


In [27]:
actions[np.argmax(res[0])]

'iloveyou'

In [28]:
actions[np.argmax(y_train[0])]

'hello'

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('action.h5')

In [29]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [30]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 38ms/step


In [31]:
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [32]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[4, 0],
        [1, 0]],

       [[2, 0],
        [3, 0]],

       [[0, 4],
        [0, 1]]], dtype=int64)

In [33]:
accuracy_score(ytrue, yhat)

0.2

In [40]:
sequence = []
sentence = []
threshold = []

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read Feed
        ret, frame = cap.read()
        #Draw Landmarks
        image, results = mediapipe_Detection(frame, holistic)
        draw_landmarks(frame,results)

        #Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[:30]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[ap.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
        
        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        #show to screen
        cv2.imshow('Webcam Feed', image)

        #end loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 22ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 18ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 22ms/step
iloveyou
1/1 [==============================] - 0s 23ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [==============================] - 0s 19ms/step
iloveyou
1/1 [==============================] - 0s 20ms/step
iloveyou
1/1 [===================

In [39]:
res[np.argmax(res)].summary

IndexError: index 5 is out of bounds for axis 0 with size 5